In [11]:
from annotation_parser import parse_annotations
from data_visualization import show_stats

PATH_ACTIONS                        = "../Data/Unprocessed/actions.xml"
PATH_ACTIONS_FILTERED               = "../Data/tmp/actions_filtered.csv"

MINIMUM_DURATION                    = 4
MINIMUM_ACTION_COUNT                = 5

In [12]:
df = parse_annotations(PATH_ACTIONS)

df["duration"] = df["end_frame"] - df["start_frame"] + 1
df_actions = df[~df["action"].isin(["ATTACK_PREPARATION", "OTHER_NO_ACTION", "OTHER_RECOVERY", 'ATTACK_FEINT'])]

merge_map = {
    "ATTACK_STOP_CUT": "ATTACK_COUNTER",
    "ATTACK_STEP_CUT": "ATTACK_LUNGE",
    "ATTACK_REMISE"  : "ATTACK_LUNGE",
    "DEFENSE_PARRY_1": "DEFENSE_PARRY",
    "DEFENSE_PARRY_2": "DEFENSE_PARRY",
    "DEFENSE_PARRY_3": "DEFENSE_PARRY",
    "DEFENSE_PARRY_4": "DEFENSE_PARRY",
    "DEFENSE_PARRY_5": "DEFENSE_PARRY",
}

df_actions_reduced = df_actions.copy()
df_actions_reduced["action"] = df["action"].apply(
    lambda x: merge_map[x] if x in merge_map else x
)

show_stats(df_actions_reduced)

action,min,max,mean,std,count
ATTACK_LUNGE,2,11,6.771300,1.640337,223
ATTACK_COUNTER,2,15,4.965909,1.784021,88
ATTACK_BEAT,3,10,5.085714,1.597267,35
ATTACK_RIPOSTE,2,11,5.058824,2.135140,17
ATTACK_FLUNGE,5,11,8.200000,2.201010,10
ATTACK_COMPOUND,5,17,10.000000,6.244998,3
ATTACK_SKY_HOOK,5,11,8.666667,3.214550,3
action,min,max,mean,std,count
DEFENSE_DISTANCE_PULL,3,30,13.969697,7.001032,66
DEFENSE_POINT_IN_LINE,3,77,29.281250,18.575451,32


Total Offensive Frames: 379
Total Defensive Frames: 128

Total Frames: 507
Total classes:  11



In [13]:
df_actions_short = df_actions_reduced[df_actions_reduced["duration"] < MINIMUM_DURATION]

df_filtered = df_actions_reduced[~df_actions_reduced["duration"].isin(df_actions_short["duration"])]
df_filtered = df_filtered[df_filtered.groupby("action")["action"].transform("size") >= MINIMUM_ACTION_COUNT]

show_stats(df_filtered)

df_filtered["file"] = df_filtered["source_id"] + "/" + df_filtered["clip_id"]
df_filtered.sort_values(by=["file", "fencer", "start_frame"], inplace=True)

df_filtered = df_filtered.reset_index(drop=True)
df_filtered["action_id"] = df_filtered.index.astype(int)

df_filtered = df_filtered[["file", "fencer", "action_id", "action", "start_frame", "end_frame"]]
df_filtered.to_csv(PATH_ACTIONS_FILTERED, index=False)

action,min,max,mean,std,count
ATTACK_LUNGE,4,11,6.880184,1.522797,217
ATTACK_COUNTER,4,15,5.302632,1.681426,76
ATTACK_BEAT,4,10,5.281250,1.528954,32
ATTACK_RIPOSTE,4,11,5.769231,1.921538,13
ATTACK_FLUNGE,5,11,8.200000,2.201010,10
action,min,max,mean,std,count
DEFENSE_DISTANCE_PULL,4,30,14.138462,6.918898,65
DEFENSE_POINT_IN_LINE,6,77,30.129032,18.242335,31
DEFENSE_PARRY,4,10,5.920000,1.411855,25


Total Offensive Frames: 348
Total Defensive Frames: 121

Total Frames: 469
Total classes:  8

